In [1]:
import pandas as pd
import numpy as np
import re
import country_converter as coco
from textab import TexTab
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
wdir = os.path.join(rootdir, '_2_intermediate', 'data')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

### overall country-level

In [3]:
df = pd.read_csv(os.path.join(wdir,'_E_ctry_religion_bch10.csv'))
df = df[['iso', 'bch10', 'major_religion', 'immg_25', 'imdwmg_25', 'n_immg_25', 'n_imdwmg_25']] 
df = df[((df.bch10 == 1980) | 
         (df.bch10 == 1990)) ]
df['xup'] = df.immg_25 * df.n_immg_25
df['xdn'] = df.imdwmg_25 * df.n_imdwmg_25
del df['immg_25'], df['imdwmg_25']
df = df[['n_immg_25', 'n_imdwmg_25', 'xup', 'xdn']].groupby([df.iso, df.bch10]).sum()
df['im_u'] = df.xup / df.n_immg_25
df['im_d'] = df.xdn / df.n_imdwmg_25
df = df[['im_u', 'im_d']].reset_index(drop=False)
df.index = [df.iso, df.bch10]
df = df.drop(['iso', 'bch10'], axis=1)
df = df.unstack().reset_index(drop=False)
df.columns = df.columns.droplevel()
df.columns = ['iso'] + ['{}_{}'.format(var, bd) 
                        for var in ['up', 'dn'] 
                        for bd in [1980, 1990]]

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'up_1980'] = df.up_1990

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'dn_1980'] = df.dn_1990

df = df.drop(['up_1990', 'dn_1990'], axis=1)
df.columns = ['iso', 'ov_up', 'ov_dn']
dfov = df.copy(deep=True)

### country-religion-level

In [4]:
df = pd.read_csv(os.path.join(wdir,'_E_ctry_religion_bch10.csv'))
df = df[['iso', 'bch10', 'major_religion', 'immg_25', 'imdwmg_25']]
df = df[((df.bch10 == 1980) | 
         (df.bch10 == 1990)) &
        ((df.major_religion == 'Christian') |
         (df.major_religion == 'Muslim') |
         (df.major_religion == 'Traditional'))].reset_index(drop=True)
df.index = [df.iso, df.bch10, df.major_religion]
df = df.drop(['iso', 'bch10', 'major_religion'], axis=1)
df = df.unstack().reset_index(drop=False)
df.columns = df.columns.droplevel()
df.columns = ['iso', 'bd', 'u_c', 'u_m', 'u_t', 'd_c', 'd_m', 'd_t']
df.index = [df.iso, df.bd]
df = df.drop(['iso', 'bd'], axis=1)
df = df.unstack().reset_index(drop=False)
df.columns = df.columns.droplevel()
df.columns = ['iso'] + ['{}_{}'.format(var, bd) 
                        for var in ['u_c', 'u_m', 'u_t', 'd_c', 'd_m', 'd_t'] 
                        for bd in [1980, 1990]]

# liberia, mali, nigeria, togo we use 1990s

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'u_c_1980'] = df.u_c_1990

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'u_m_1980'] = df.u_m_1990

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'u_t_1980'] = df.u_t_1990

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'd_c_1980'] = df.d_c_1990

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'd_m_1980'] = df.d_m_1990

df.loc[(df.iso == 'LBR') |
       (df.iso == 'MLI') |
       (df.iso == 'NGA') |
       (df.iso == 'TGO'), 'd_t_1980'] = df.d_t_1990

dfc = df.drop(['u_c_1990', 'u_m_1990', 'u_t_1990', 'd_c_1990', 'd_m_1990', 'd_t_1990'], axis=1)

### putting together and outputting table

In [5]:
def add_colorbars(x1, x2):
    
    if pd.isnull(x1) or pd.isnull(x2):
        return ''
    elif x1 > x2:
        width=x1-x2
        return '\colorbox{white}{\parbox{%sem}{\hfill}}\colorbox{Green}{\parbox{%sem}{\hfill}}' % (5*width, 5*width)
    else:
        width=x2-x1
        return '\colorbox{Red}{\parbox{%sem}{\hfill}}\colorbox{white}{\parbox{%sem}{\hfill}}' % (5*width, 5*width)

In [6]:
df = pd.merge(dfov, dfc, on=['iso'], how='outer')
df['country'] =  coco.convert(names=list(df['iso']), to='name_short')
del df['iso']

In [7]:
df['diff_up'] = df.apply(lambda x: add_colorbars(x['u_c_1980'], x['u_m_1980']), axis=1)
df['diff_dn'] = df.apply(lambda x: add_colorbars(x['d_c_1980'], x['d_m_1980']), axis=1)

In [8]:
df = df[['country', 
         'ov_up', 'u_c_1980', 'u_m_1980', 'u_t_1980', 'diff_up', 
         'ov_dn', 'd_c_1980', 'd_m_1980', 'd_t_1980', 'diff_dn']].copy()
df['aux'] = df.u_c_1980 - df.u_m_1980
df = df.sort_values(by='aux', ascending=False).reset_index(drop=True).copy(deep=True)
del df['aux']

In [9]:
for col in df.columns[1:]:
    if 'diff' not in col:
        df[col] = np.round(df[col], 3)
for col in df.columns:
    df[col] = df[col].astype('str')
df = df.replace('nan', '')

In [10]:
tt = TexTab(df)

In [11]:
cns = ['', '(1)', '(2)', '(3)', '(4)', '(5)', '(6)', '(7)', '(8)', '(9)', '(10)']
lcols = ['country']
gaps = ['country', 'diff_up']
cap = "Country-group-level estimates of IM, ages 14-25"
lab = "tab:_im_ctry_majrel_1425"
lines = []
notes = '''This table reports upward and downward IM measures for the 1980s cohort (the cohort with the 
broadest coverage) for individuals aged 14-25 by country and major religious group. Because of the timing of censues, 
we have to use the 1990s cohort for Liberia, Mali, Nigeria, and Togo as for those countries we 
don't have 14-25 year olds born in the 1980s. Columns (1) - (4) show the estimates for country-cohort-level upward IM,
columns (6) - (9) those for downward IM. Columns (1) and (6) give the country-level estimates across all groups,
columns (2) - (4) and (7) - (9) give estimates by group. Columns (5) and (10) show differences
between Christians and Muslims. Red bars indicate a negative difference, green bars a positive
difference.
'''

In [12]:
tab = tt.create_tab_onepanel(cns=cns, gaps=gaps, lcols=lcols,
                             cap=cap, lab=lab, lines=lines, notes=notes, clines=False)

In [13]:
nl = '\\\\multicolumn{1}{l}{}&&\\\\multicolumn{5}{c}{upward IM}&&\\\\multicolumn{5}{c}{downward IM}'
tab = re.sub(r'&&&&&&&&&&&', nl, tab)
nl = 'country&&overall&Christian&Muslim&Traditional&$\\\\Delta(c-m)$&&overall&Christian&Muslim&Traditional&$\\\\Delta(c-m)$\\\\\\\n'
tab = re.sub(r'(\\hline\nNigeria)', r'%s\1' % nl, tab)
tab = re.sub(r'\(5\)&\(6\)', '(5)&&(6)', tab)
tab = re.sub(r'lccccccccccc', r'lcccccccccccc', tab)

In [14]:
fh = open(outdir + "/_2_ctry_im_majrel_1425.tex", "w")
fh.write(tab)
fh.close()